# Linear Regression

Hi Friends!! 

The main intension here is to provide steps on how a Linear Regression can be performed

This is purely based on my undestanding

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings("ignore")

##### Problem Statement: on the basis of the expenditure incurred on each type of advertisement we need to predict the increse in sales                  

### Understanding Data

In [ ]:
data = pd.read_csv(r"../input/advertising-data/Advertising.csv",index_col=0,header=0)
data.head()

In [ ]:
print(data.dtypes,"\n")
print(data.shape,"\n")

Dependent/Target Variable : Sales, dtype as float(continuous) 

No predictor is of data type Object

Model : Linear Regression

### Steps :

1. Pre model building Assumptions(Since Linear Regression)

2. EDA and Data Cleanning

3. Model building - Linear Regression

4. Post Model building Assumptions

5. Ridge Model

6. Lasso Model

7. Conclusion

### Step 1: Pre Model Buiding Assumptions

In [ ]:
#Assumption 1 - No outliers in the data
#can be checked using box plot
f,axes=plt.subplots(2,2,figsize=(7,7))
sns.boxplot(y="TV",data=data,ax=axes[0,0],color="orange")
sns.boxplot(y="Radio",data=data,ax=axes[0,1],color="pink")
sns.boxplot(y="Newspaper",data=data,ax=axes[1,0],color="grey")
sns.boxplot(y="Sales",data=data,ax=axes[1,1],color="brown")

From the above plots we can see that only Newspaper has Outliers

In [ ]:
#Assumption 2 (Assumption of Linearity):
#Every independent variable should have a linear relationship with dependent variable

sns.pairplot(data,x_vars=["TV","Radio","Newspaper"],y_vars="Sales",kind="reg")

TV vs. sales - strong positive linear relatioship

Radio vs. sales - weak positive relationship(comparitively)

Newspaper vs. sales - no relationship(we can discard such kind of variables which donot have a linear relationship)

In [ ]:
#Assumption 3:
#The dependent variable should follow an approx. normal distribution
#can be checked using a distplot

sns.distplot(data.Sales,bins=10,color="green",hist=True)

The above distribution plot for Sales is approximately normal

In case if the data is not normally distributed - we need to tranform data using log func or square root func

Log Tranformation - since it smoothens the peaks

import numpy as np

Y_log = np.log(Y)

sns.distplot(Y_log,hist=True)

In [ ]:
#Assumption 4-There should no multi collinearity in the data
#Can be checked using correlation and VIF(Variance inflation factor)
#Relationship between the independent variables should not exsit

ind = data[["TV","Radio","Newspaper"]]

corr_df = ind.corr(method = "pearson")
print(corr_df,"\n")

sns.heatmap(corr_df,vmax =1.0,vmin=-1.0,annot=True)


+1 : positively correlated

-1 : negatively correlated

0 : no correlation

We can see that there is no multi collinearity amng independent/predictor variables

If we have any multi collinearity :

Technique 1- drop either of them

Technique 2-check the correlation values of both variables with all the other values and drop that one which has high collinearity values with the others

But always these techniques might not help - so we go with VIF(Variance Inflation Factor), for more accurate results

In [ ]:
#VIF - 1/(1-R^2)
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

vif_df = pd.DataFrame()
vif_df["features"] = ind.columns
vif_df["VIF Factor"] = [vif(ind.values, i) for i in range(ind.shape[1])]
vif_df.round(2)

### Step 2 : EDA and Data Cleaning

In [ ]:
data.info()
#No Null's in the data

In [ ]:
data.describe()

In [ ]:
data.boxplot(column="Newspaper")

In [ ]:
data.hist(bins = 20,figsize=(10,7))

It is a good practise to check skewness in data

Newspaper is highly skewed

If the skewness is not acceptable - transform the data

In [ ]:
#Transforming data based on Skewness

from scipy.stats import skew
data_num_skew = data.apply(lambda x: skew(x.dropna()))
data_num_skewed = data_num_skew[(data_num_skew > .75) | (data_num_skew < -.75)]

print(data_num_skew,"\n")
print(data_num_skewed)

import numpy as np
data[data_num_skewed.index] = np.log1p(data[data_num_skewed.index])


In [ ]:
data.hist(bins=20)

The transformation didn't make much difference, Newspaper might not be a good variable for the model

In [ ]:
df = data.copy()
df.shape

### Step 3 : Model Building(Linear Regression)

In [ ]:
#Creating x and Y:
X=data[["TV","Radio","Newspaper"]]
Y=data["Sales"]
print(X.head(),"\n")
print(Y.head())

### Using sklearn

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =0.2,random_state=10)

In [ ]:
print(X_train.shape,"\n")
print(Y_train.shape,"\n")
print(X_test.shape,"\n")
print(Y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()

lm.fit(X_train,Y_train)

print(lm.intercept_,"\n")
print(list(zip(X.columns,lm.coef_)))

In [ ]:
#Prediction
Y_pred = lm.predict(X_test)
print(Y_pred)

In [ ]:
#EVALUATING model manually, just to cross check sice the data is very small
#by comparing the actual Y values and predicted Y values

new_df = pd.DataFrame()
new_df =X_test

new_df["Actual Sales"] =Y_test
new_df["Predicted Sales"] = Y_pred
new_df

By comparing the values from the above data frame we can say that model is a good one; but how good?

So we need to pass our model through some performance metrics -  R^2, adjusted R^2, RMSE, p-value, VIF(we have already calculated the values above)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error

import numpy as np

r2 = r2_score(Y_test,Y_pred)
print("R^2 = ",r2,"\n")

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1) #formula for adjusted R2
print("Adjusted R^2 = ",adjusted_r_squared,"\n")

rmse = np.sqrt(mean_squared_error(Y_test,Y_pred))#sqrt(SME) = RMSE
print("RMSE = ",rmse,"\n")

#Checking whether RMSE is a good value or not

per=str(round(((rmse/max(Y_test))*100),2)) + "%"
print("Y_min = ",min(Y_test),",  Y_max = ",max(Y_test), ", Percentage of Errors :", per)

We got good value for R2(towards 1)

Adjusted R^2 will always be slightly lower that R^2

Value for RMSE is also good since it is closer to Y_min value thus the errors are neglectable (10%)

### Using statsmodel

We can get p-values only from statsmodel

In [ ]:
new_df = pd.DataFrame()
new_df = X_train

new_df["Sales"] = Y_train
new_df.shape

In [ ]:
import statsmodels.formula.api as sm

lm_model = sm.ols(formula="Sales ~ TV + Radio + Newspaper",data=new_df).fit()

print(lm_model.params,"\n")
print(lm_model.summary())

#Since p-value of Newspaper > 0.05, we can drop it

AIC(Akile Information Criteria) - when we have to compare different models with same algorithm. When the variables are added or deleted we will get different AIC's for every modep built - lowest AIC give the best model 

BIC(Bayes Information Criteria)

p-value => logic is hypothesis testing

H0:There is no significant relationship between X and Y

H1:There is a significant relationship between X and Y

Threshold by default 0.05

If the p-val is <= 0.05(less), then Reject H0

If the p-val is > 0.05(High), then Select H0

Drop the variable which have p values high(> 0.05)

 

In [ ]:
Y_pred_new = lm_model.predict(X_test)

from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2 = r2_score(Y_test,Y_pred_new)
print("R^2 :",r2,"\n")#good value of R2(towards 1)

rmse = np.sqrt(mean_squared_error(Y_test,Y_pred_new))#sqrt(SME) => RMSE
print("RMSE :", rmse,"\n")

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)#formula for adjusted R2
print("Adjusted R^2 :",adjusted_r_squared)

Model after Newspaper is removed from the formula

In [ ]:
import statsmodels.formula.api as sm

lm_model = sm.ols(formula="Sales ~ TV + Radio",data=new_df).fit()

print(lm_model.params,"\n")
print(lm_model.summary())

In [ ]:
Y_pred_new = lm_model.predict(X_test)

from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2 = r2_score(Y_test,Y_pred_new)
print("R^2 :",r2,"\n")#good value of R2(towards 1)

rmse = np.sqrt(mean_squared_error(Y_test,Y_pred_new))#sqrt(SME) => RMSE
print("RMSE :", rmse,"\n")

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)#formula for adjusted R2
print("Adjusted R^2 :",adjusted_r_squared)

We can notice that R^2 has been slightly increased after removing Newspaper from the model 

### Step 4 : Post Model Building Assumptions

In [ ]:
#Assumption 5 : No Auto Correlation in the data
#Can be checked by Durbin Watson Test

#Threshold - [0,4]
#close to 2 - no auto correlation
#close to 0 - positive auto correlation
#close to 4 - negative auto correlation

#From the above Summary DWT = 2.1 (No Auto Correlation)

In [ ]:
#Assumption 6 :Errors should be random 
#Can be checked using Residuals(Errors) Vs. Fitted values(actual) plot

plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

model_fitted_y = lm_model.fittedvalues

plot_lm_1.axes[0] = sns.residplot(model_fitted_y, 'Sales', data=new_df, lowess=True)

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')

Interpreting the graph :

1. We need a random line or a cure

2. It should not follow a cyclic pattern, or a line/curve like pattern - if so then the errors are not random

In [ ]:
#Assumption 7: Errors should follow an approximate normal distribution
#can be checked using the normal Quantile-Quantile plot (normal qq plot)


res = lm_model.resid
import statsmodels.api as stm
import scipy.stats as stats
fig = stm.qqplot(res, fit=True, line='45')
plt.title('Normal Q-Q')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Standardized Residuals')
plt.show() 

Interpreting the graph:

If the error points are falling on the 45 degree red line then we can say that the errors are normally distributed

In [ ]:
#Assumption 8 : Errors should follow a constant variance (Homoskedasticity)
#Can be checked using the scale location plot

# normalized residuals
model_norm_residuals = lm_model.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))

plot_lm_3 = plt.figure(3)
plot_lm_3.set_figheight(8)
plot_lm_3.set_figwidth(12)
plt.scatter(model_fitted_y, model_norm_residuals_abs_sqrt, alpha=0.5)
sns.regplot(model_fitted_y, model_norm_residuals_abs_sqrt, lowess=True)


plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$')

Interpreting the graph :

1. When we get an approximately horizontal line on the graph we can say that errors are following a constant variance

2. There can be a little curve, we dont want a steep curve

3. When we consider any part of the plot and find the variance it should be the same(approx.)

### Step 5 : Ridge Model

1. RIDGE MODEL(L2 panelty)

2. Ridge just adjusts the b coefficients

3. When compared to the previous model the coefficient values have changed slightly

In [ ]:
X=df[["TV","Radio","Newspaper"]]
Y=df["Sales"]
print(X.head(),"\n")
print(Y.head())

#Newspaper variable is considered just to understand Ridge Model

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =0.2,random_state =10)

In [ ]:
print(X_train.shape,"\n")
print(X_test.shape,"\n")
print(Y_train.shape,"\n")
print(Y_test.shape,"\n")

In [ ]:
from sklearn.linear_model import Ridge

lm = Ridge()
lm.fit(X_train,Y_train)

print(lm.intercept_,"\n")
print(lm.coef_)


The coefficient of the variale which is not contributing more to the model will be the least

Coefficient of Newspaper = -0.048(least)

### Step 6 : Lasso Model

1. LASSO MODEL(L1 panelty)

2. Lasso zero's down the b coefficients whose variables are insignificant - can be used as feature extraction

3. When compared to the Ridge model Lasso has slight change in the coefficient values 

In [ ]:
from sklearn.linear_model import Lasso

lm = Lasso()
lm.fit(X_train,Y_train)

print(lm.intercept_,"\n")
print(lm.coef_)


Since Newspaper is not contributing to the model it's coefficient is replaced with "0" 

### Step 7 : Conclusion

1. R^2 =  0.8354496662944217 (good value)

2. Adjusted R^2 =  0.8337791045309133 (good value)

3. RMSE =  2.5878817077378105 (good value w.r.t. the minimum and maximum values of target variable)

So we can conclude that the Linear Regression model built is a good one.  

In [ ]:
test_sales = X_test.copy()

test_sales["Sales"] = Y_pred
test_sales["Predicted_Sales"] = round(test_sales.Sales,2)
test_sales.drop("Sales",axis=1,inplace=True)
test_sales.head()